### Prediction with trained model

1. Loads a model
2. Load weights from a checkpoint
3. Load zarr dataset for prediction (patch generation is handled by this class)
4. Perform prediction
5. Perform checks on subtiles
5. Write results to csv

In [1]:
import torch
def gpu_mem_stats():
    t = torch.cuda.get_device_properties(0).total_memory
    c = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    print(f'{t/(1024**3):0.2f} Gb Total memory')
    print(f'{c/(1024**3):0.2f} Gb cached memory')
    print(f'{a/(1024**3):0.2f} Gb allocated memory')
    return

In [2]:
import sys
from pathlib import Path

#Add path to parent folder for imports
sys.path.append(str(Path.cwd().parent))

import torch
from torch.utils.data import DataLoader
from utils.data import Pred_Ai224_RG_Zarr,Pred_Sampler_Zarr
from models.unet import Ai224_RG_UNet
from utils.analysis import pred_to_xy
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#Load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tensor = lambda x: torch.tensor(x).to(dtype=torch.float32).to(device)
tensor_ = lambda x: torch.as_tensor(x).to(dtype=torch.float32).to(device)
tonumpy = lambda x: x.cpu().detach().numpy()

model = Ai224_RG_UNet()
ckpt_file = '../../../dat/Ai224_RG_models/CE_wt244_Adam_run_v3_norm/44587_ckpt.pt'
checkpoint = torch.load(ckpt_file,map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
loss = checkpoint['loss']

model.to(device)
print(f'Validation loss was {loss:0.8f}')

Validation loss was 1.10335231


In [3]:
gpu_mem_stats()

10.92 Gb Total memory
0.01 Gb cached memory
0.01 Gb allocated memory


In [4]:
patch_size=260
output_size = 172
scale = 255.
n_x_patch_per_subtile = 10
n_y_patch_per_subtile = 10
batch_size = n_x_patch_per_subtile*n_y_patch_per_subtile

im_path = '/home/rohan/Remote-AI-root/allen/programs/celltypes/workgroups/mct-t200/Molecular_Genetics_Daigle_Team/Elyse/Unet_WB_testing/547656/'
fname = '0539050069-0070.zarr'
csv_path = '/home/rohan/Remote-AI-root/allen/programs/celltypes/workgroups/mct-t200/Molecular_Genetics_Daigle_Team/Elyse/Unet_WB_testing/547656/csv/'
csv_path = './'
csv_fname_g = fname.replace('.zarr','_g.csv')
csv_fname_r = fname.replace('.zarr','_r.csv')

pred_dataset = Pred_Ai224_RG_Zarr(patch_size = patch_size,
                             output_size = output_size,
                             im_path=im_path,
                             fname=fname,
                             scale=scale)

pred_sampler = Pred_Sampler_Zarr(dataset=pred_dataset, 
                                 n_x_patch_per_subtile=n_x_patch_per_subtile, 
                                 n_y_patch_per_subtile=n_y_patch_per_subtile)

pred_dataloader = DataLoader(pred_dataset, batch_size=batch_size, shuffle=False,
                            sampler=pred_sampler, drop_last=False, pin_memory=True)

pred_datagen = iter(pred_dataloader)

In [5]:
#Empty subtile for labels:
n_labels = 3
n_channels = 2
x_size = pred_dataset.output_size*pred_sampler.n_x_patch_per_subtile
y_size = pred_dataset.output_size*pred_sampler.n_y_patch_per_subtile
output_size = pred_dataset.output_size
offset = int((pred_dataset.patch_size - pred_dataset.output_size)/2)

In [6]:
pred_g = np.empty(shape=[n_labels,x_size, y_size],dtype=float)
pred_r = np.empty(shape=[n_labels,x_size, y_size],dtype=float)

input_im = np.empty(shape=[n_channels,x_size, y_size],dtype=float)
g_list,r_list = [],[]

#Set model in eval mode
model.eval()

for i in tqdm(range(len(pred_datagen))):
    batch = next(pred_datagen)
    with torch.no_grad():
        xg,xr,_,_ = model(tensor_(batch['im']))
    xg = tonumpy(xg)
    xr = tonumpy(xr)
    
    for j in range(batch['idx'].shape[0]):
        subtile_ind = tonumpy(batch['idx'][0])
        ind = tonumpy(batch['idx'][j]) - subtile_ind
        input_im[:,ind[0]:ind[0]+output_size,ind[1]:ind[1]+output_size] = tonumpy(batch['im'][j])[:,offset:-offset,offset:-offset]
        pred_g[:,ind[0]:ind[0]+output_size,ind[1]:ind[1]+output_size] = xg[j]
        pred_r[:,ind[0]:ind[0]+output_size,ind[1]:ind[1]+output_size] = xr[j]
    
    del xg,xr,batch
    com_g,n_elem_g = pred_to_xy(fg=np.squeeze(pred_g[2,:,:]),bo=np.squeeze(pred_g[1,:,:]),pred_thr=0.5,n_elem_thr=5)
    com_r,n_elem_r = pred_to_xy(fg=np.squeeze(pred_r[2,:,:]),bo=np.squeeze(pred_r[1,:,:]),pred_thr=0.5,n_elem_thr=5)
    
    #Convert patch co-ordinates into global co-ordinates
    global_com_g = com_g + subtile_ind.reshape(1,2) + offset
    global_com_r = com_r + subtile_ind.reshape(1,2) + offset
    g_list.append(global_com_g)
    r_list.append(global_com_r)
    
    #Write to file
    df_g = pd.DataFrame({'x':global_com_g[:,0],'y':global_com_g[:,1],'n':n_elem_g})
    df_r = pd.DataFrame({'x':global_com_r[:,0],'y':global_com_r[:,1],'n':n_elem_r})
    
    
    if i>0:
        df_g.to_csv(csv_fname_g, mode='a', header=False, index=False)
        df_r.to_csv(csv_fname_r, mode='a', header=False, index=False)
    else:
        df_g.to_csv(csv_fname_g, mode='w', header=True, index=False)
        df_r.to_csv(csv_fname_r, mode='w', header=True, index=False)

com_g = np.concatenate(g_list,axis=0)
com_r = np.concatenate(r_list,axis=0)

100%|██████████| 9/9 [00:17<00:00,  1.99s/it]


In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(np.squeeze(pred_dataset.IM[:,:,0])/(2**16),vmin=0,vmax=0.005)
plt.plot(com_g[:,1],com_g[:,0],'x',color='#ffffff')
plt.plot(com_r[:,1],com_r[:,0],'o',color='#00ff00',markerfacecolor='None')
plt.grid(False)
plt.show()

#### Remove potential duplicates from entire tile
 - These would mostly occur along shared edges of subtiles. 

In [ ]:
# #TODO
# from sklearn.metrics import pairwise_distances
# d = pairwise_distances(X=com_gt, Y=com, metric='euclidean')
# ind_tp_in_gt = np.any(d<cell_radius,axis=1)
# ind_tp_in_pred = np.any(d<cell_radius,axis=0)

In [19]:
x = Path.iterdir()
(im_path,'*.zarr')

AttributeError: type object 'Path' has no attribute 'listdir'

In [30]:
x = Path.iterdir(im_path)

In [31]:
for i in x: print(i)

AttributeError: 'str' object has no attribute '_closed'

In [41]:
Path.is_file('/home/rohan/Remote-AI-root/allen/programs/celltypes/workgroups/mct-t200/Molecular_Genetics_Daigle_Team/Elyse/Unet_WB_testing/547656/0539050069-0001.zarr')

AttributeError: 'str' object has no attribute 'stat'

In [40]:
im_path+'0539050069-0001.zarr'

'/home/rohan/Remote-AI-root/allen/programs/celltypes/workgroups/mct-t200/Molecular_Genetics_Daigle_Team/Elyse/Unet_WB_testing/547656/0539050069-0001.zarr'

In [43]:
Path.is_file('./prediction.ipynb')

AttributeError: 'str' object has no attribute 'stat'

In [48]:
x = Path(im_path).glob('**/*')

In [50]:
a = next(x)

In [53]:
a.is_dir()

True

In [59]:
fname_list

['0539050069-0001.zarr',
 '0539050069-0002.zarr',
 '0539050069-0003.zarr',
 '0539050069-0004.zarr',
 '0539050069-0005.zarr',
 '0539050069-0006.zarr',
 '0539050069-0007.zarr',
 '0539050069-0008.zarr',
 '0539050069-0009.zarr',
 '0539050069-0010.zarr',
 '0539050069-0011.zarr',
 '0539050069-0012.zarr',
 '0539050069-0013.zarr',
 '0539050069-0014.zarr',
 '0539050069-0015.zarr',
 '0539050069-0016.zarr',
 '0539050069-0017.zarr',
 '0539050069-0018.zarr',
 '0539050069-0019.zarr',
 '0539050069-0020.zarr',
 '0539050069-0021.zarr',
 '0539050069-0022.zarr',
 '0539050069-0023.zarr',
 '0539050069-0024.zarr',
 '0539050069-0025.zarr',
 '0539050069-0026.zarr',
 '0539050069-0027.zarr',
 '0539050069-0028.zarr',
 '0539050069-0029.zarr',
 '0539050069-0030.zarr',
 '0539050069-0031.zarr',
 '0539050069-0032.zarr',
 '0539050069-0033.zarr',
 '0539050069-0034.zarr',
 '0539050069-0035.zarr',
 '0539050069-0036.zarr',
 '0539050069-0037.zarr',
 '0539050069-0038.zarr',
 '0539050069-0039.zarr',
 '0539050069-0040.zarr',
